In [0]:
import datetime
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord@etsy.com",
        "phone_numbers": ["+0 000 000 000", "+0 111 111 111"],
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021,1,15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "brewitt@etsy.com",
        "phone_numbers": ["+1 000 000 000", "+1 111 111 111"],
        "is_customer": True,
        "amount_paid": 900.55,
        "customer_from": datetime.date(2021,2,15),
        "last_updated_ts": datetime.datetime(2021, 2, 17, 1, 15, 0)
    },
    {
        "id": 3,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "brewitt@etsy.com",
        "phone_numbers": None,
        "is_customer": True,
        "amount_paid": 900.55,
        "customer_from": datetime.date(2021,2,15),
        "last_updated_ts": datetime.datetime(2021, 2, 17, 1, 15, 0)
    }
]


In [0]:
from pyspark.sql import Row

In [0]:
users_df = spark.createDataFrame([Row(**user) for user in users])

In [0]:
users_df.printSchema()

root
-- id: long (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- email: string (nullable = true)
-- phone_numbers: array (nullable = true)
 |-- element: string (containsNull = true)
-- is_customer: boolean (nullable = true)
-- amount_paid: double (nullable = true)
-- customer_from: date (nullable = true)
-- last_updated_ts: timestamp (nullable = true)

In [0]:
users_df.show()

+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
 id|first_name| last_name| email| phone_numbers|is_customer|amount_paid|customer_from| last_updated_ts|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
 1| Corrie|Van den Oord|cvandenoord@etsy.com|[+0 000 000 000, ...| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|
 2| Nikolaus| Brewitt| brewitt@etsy.com|[+1 000 000 000, ...| true| 900.55| 2021-02-15|2021-02-17 01:15:00|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+

In [0]:
users_df.select("id", "phone_numbers").show(truncate=False)

+---+--------------------------------+
id |phone_numbers |
+---+--------------------------------+
1 |[+0 000 000 000, +0 111 111 111]|
2 |[+1 000 000 000, +1 111 111 111]|
+---+--------------------------------+

In [0]:
users_df.columns

Out[21]: ['id',
 'first_name',
 'last_name',
 'email',
 'phone_numbers',
 'is_customer',
 'amount_paid',
 'customer_from',
 'last_updated_ts']

In [0]:
users_df.dtypes

Out[23]: [('id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone_numbers', 'array<string>'),
 ('is_customer', 'boolean'),
 ('amount_paid', 'double'),
 ('customer_from', 'date'),
 ('last_updated_ts', 'timestamp')]

In [0]:
from pyspark.sql.functions import explode

In [0]:
# withColumn adds a column
# explode convert one line to multi lines
users_df. \
    withColumn("phone_number", explode("phone_numbers")). \
    drop("phone_numbers"). \
    show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+--------------+
 id|first_name| last_name| email|is_customer|amount_paid|customer_from| last_updated_ts| phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+--------------+
 1| Corrie|Van den Oord|cvandenoord@etsy.com| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|+0 000 000 000|
 1| Corrie|Van den Oord|cvandenoord@etsy.com| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|+0 111 111 111|
 2| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00|+1 000 000 000|
 2| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00|+1 111 111 111|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+--------------+

In [0]:
from pyspark.sql.functions import col

In [0]:
users_df.\
    select("id", col("phone_numbers")[0].alias("mobile"), col("phone_numbers")[1].alias("home")).\
    show()

+---+--------------+--------------+
 id| mobile| home|
+---+--------------+--------------+
 1|+0 000 000 000|+0 111 111 111|
 2|+1 000 000 000|+1 111 111 111|
+---+--------------+--------------+

In [0]:
from pyspark.sql.functions import explode_outer

In [0]:
# explode_outer produces null values unlike explode
users_df. \
    withColumn("phone_number", explode_outer("phone_numbers")). \
    drop("phone_numbers"). \
    show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+--------------+
 id|first_name| last_name| email|is_customer|amount_paid|customer_from| last_updated_ts| phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+--------------+
 1| Corrie|Van den Oord|cvandenoord@etsy.com| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|+0 000 000 000|
 1| Corrie|Van den Oord|cvandenoord@etsy.com| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|+0 111 111 111|
 2| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00|+1 000 000 000|
 2| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00|+1 111 111 111|
 3| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00| null|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+--------------+